# Infosys Quarterly Report Analysis
In this demo, we explore answering complex queries by decomposing them into simpler sub-queries.

In [1]:
!pip install getpass4 pypdf llama-index openai faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.5/966.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 28.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
  Created wheel for clipboard: filename=clipboard-0.0.4-py3-none-any

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/LLM/LLMWorkshop/materials/RAG/ifrs-inr-press-release.pdf

/content/drive/MyDrive/LLM/LLMWorkshop/materials/RAG/ifrs-inr-press-release.pdf


In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
import os
from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass("OpenAI Key:")

OpenAI Key:··········


In [6]:
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex, StorageContext
from llama_index.response.pprint_utils import pprint_response
from llama_index.llms import OpenAI

from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

In [7]:
import faiss
from llama_index.vector_stores import FaissVectorStore

## Configure LLM service

In [8]:
llm = OpenAI(temperature=0,
             model="text-davinci-003",
             max_tokens=500)

service_context = ServiceContext.from_defaults(llm=llm)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Load data
Downloaded from

https://www.infosys.com/investors/reports-filings/quarterly-results.html

In [10]:
q1_2024 = SimpleDirectoryReader(
    input_files=["/content/drive/MyDrive/LLM/LLMWorkshop/materials/RAG/ifrs-inr-press-release.pdf"]
).load_data()

# Build indices

In [11]:
# dimensions of text-ada-embedding-002
d = 1536
faiss_index = faiss.IndexFlatL2(d)

In [12]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
q1_2024_index = VectorStoreIndex.from_documents(q1_2024, storage_context=storage_context)

## Build query engines

In [26]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [27]:
q1_2024__engine = q1_2024_index.as_query_engine(similarity_top_k=2, verbose = True)

## Run queries

In [28]:
response = q1_2024__engine.query(
    "What is the revenue growth for the quarter?"
)

In [15]:
print(response)

The revenue growth for the quarter is 4.2%.


In [16]:
response = q1_2024__engine.query("Which are some of the key customer wins?")

In [17]:
print(response)

Danske Bank, bp, Aramco, British Telecom, Walmart Commerce Technologies, vidaXL, LexisNexis, ATP, Keytrade Bank, and XacBank are some of the key customer wins mentioned in the context.


In [18]:
response = q1_2024__engine.query("What are some of the key achievements in the quarter?")

In [19]:
print(response)

Some of the key achievements in the quarter include winning awards such as the MEA Finance 'Best Composable Banking Technology Solution Provider' award and the SS&C Blue Prism Partner Excellence Awards in multiple categories. Infosys was also recognized as a leader in various assessments, including Low-Code Application Development Services, Microsoft Dynamics 365 Services, Application Automation Services, 5G Engineering Services, Wealth and Asset Management, Adobe Experience Cloud Services, SAP Cloud Migration Services, and Quality Engineering. Additionally, Infosys was positioned as a leader in HFS Horizons reports, IDC MarketScape assessments, and ISG Provider Lens studies.


In [20]:
response = q1_2024__engine.query("What is the total asset at end of june 2023?")

In [21]:
print(response)

The total assets at the end of June 2023 is 131,322 crore.


In [22]:
response = q1_2024__engine.query("What is significant changes in the item in balance sheet in june 2023 from march 2023?")

In [23]:
print(response)

The significant changes in the balance sheet item from June 2023 compared to March 2023 are as follows:

1. Cash and cash equivalents increased from 12,173 crore to 12,310 crore.
2. Earmarked bank balance for dividend increased from 0 crore to 7,262 crore.
3. Current investments decreased from 6,909 crore to 5,536 crore.
4. Trade receivables increased from 25,424 crore to 26,183 crore.
5. Unbilled revenue decreased from 15,289 crore to 14,628 crore.
6. Other current assets increased from 11,086 crore to 11,555 crore.
7. Property, plant and equipment and Right-of-use assets decreased from 20,675 crore to 20,487 crore.
8. Goodwill and other Intangible assets decreased from 8,997 crore to 8,876 crore.
9. Non-current investments decreased from 12,569 crore to 11,991 crore.
10. Unbilled revenue decreased from 1,449 crore to 1,379 crore.
11. Other non-current assets decreased from 11,245 crore to 11,115 crore.
12. Trade payables decreased from 3,865 crore to 3,759 crore.
13. Unearned revenue